# Landscaping Startups in Portland, Oregon, United States

### Business Problem

The neighborhoods in the city of Portland often have a variety of natural and artificial landscapes that need stewardship and maintenance. Venues may include parks, garden centers, country clubs, florists, hardware stores, private residences, and any business that caters to creating, maintaining, and managing vegetation and landscapes. 

A hypothetical group of investors is looking to start a landscaping business that maintains, desings, and installs vegetation in public and private spaces. It would behoove this group of stakeholders to determine in which neighbourhoods they find the most amount of these locations. 

This information would allow them to decide where they should place their new landscaping business; such that travel times between clients and appointments is reduced and their ability to reach customers improves. On the other hand, venue category may provide useful intelligence for deciding which neighborhood is closest to businesses the sell the inputs the startup will need more ocassionally: seed, plants, fertilizer, pesticides, or similar hardware.  Likewise, results may also allow them to determine in which neighborhoods they should spend resources researching the market and possible competitors.

![This landscape could use some mowing](https://img.timeinc.net/time/2010/portland_tavel/portland_park.jpg)

<p style="text-align: center;">A park in the city of Portland that could use a mowing.</p>

### Data

We will use Foursquare location data to access venue categories in Portland. We will determine neighborhood locations in the city of Portland by scraping several lists from the [Neighborhoods of Portland, Oregon](https://portlandneighborhood.com/portland-neighborhoods) site. The list will be sorted and cleaned so that we can parse it to the geocoder and determine geographical locations for each neighborhood. Following this, we will explore each neighborhood within a 1000 m radius for venues. We will categorize the venues and determine which categories are most related to landscaping and gardening businesses. We will classify each neighborhood according to their rate of landscaping-related businesses. This will allow us to make a recommendation for which neighborhoods the investors should pay closer attention to.

## 1. Importing packages

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


from bs4 import BeautifulSoup # We use beautiful soup to scrape the website from Portland's local government site



print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

In [73]:
site = 'https://portlandneighborhood.com/portland-neighborhoods'
nhs = requests.get(site)
soup = BeautifulSoup(nhs.text, "html.parser")

nhs_list = soup.find_all("a")
nhs_list[12:22]

portland = []

for child, num in zip(nhs_list, enumerate(nhs_list)): # For loop produces the strings in all the 'a' tags of our soup
    portland.append(child.string)
    print(num[0], child.string) # the row number tells us where to slice. We know from the website where the list starts and ends.
    


0 None
1 None
2 None
3 None
4 None
5 None
6 None
7 Events
8 News
9 None
10 Go here
11 Portland zip codes
12 Arbor Lodge
13 Boise & Historic Mississippi Avenue
14 Bridgeton
15 Cathedral Park
16 East Columbia
17 Eliot
18 Hayden Island
19 Humboldt
20 Kenton
21 Overlook
22 Piedmont
23 Portsmouth
24 St. Johns
25 University Park
26 Arlington Heights
27 Forest Park
28 Goose Hollow
29 Hillside
30 Linnton
31 Northwest Heights
32 NW 21st & 23rd Avenue – Nob Hill
33 Sylvan Highlands
34 Alameda
35 Alberta Arts District
36 Beaumont-Wilshire
37 North Tabor
38 Concordia
39 Cully
40 Grant Park
41 Hollywood
42 Irvington
43 King
44 Madison South
45 Rose City Park
46 Roseway
47 Sabin
48 Sullivan’s Gulch
49 Sumner
50 Vernon
51 Woodlawn
52 China Town/Old Town
53 Downtown
54 Lloyd Center
55 Pearl District
56 South Waterfront
57 Arnold Creek
58 Ashcreek
59 Bridlemile
60 Collins View
61 Far Southwest
62 Hayhurst
63 Healy Heights
64 Hillsdale
65 Homestead
66 Maplewood
67 Markham
68 Marshall Park
69 Multnomah
7

In [76]:
portland_nhs = portland[12:105]
portland_nhs = pd.DataFrame(data = {'Neighborhood':portland_nhs})
portland_nhs

,Neighborhood
0,Arbor Lodge
1,Boise & Historic Mississippi Avenue
2,Bridgeton
3,Cathedral Park
4,East Columbia
5,Eliot
6,Hayden Island
7,Humboldt
8,Kenton
9,Overlook


In [77]:
portland_nhs[1] = 'Boise'
portland_nhs[20] = 'Nob Hill'
portland_nhs[40] = 'Old Town'